In [43]:
import datetime
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import covariance, cluster
#from matplotlib.finance import quotes_historical_yahoo_ochl as quotes_yahoo

from matplotlib.dates import (MONDAY, DateFormatter, MonthLocator, WeekdayLocator, date2num)
from mpl_finance import plot_day_summary_oclh

# Input file containing company symbols 
input_file = 'company_symbol_mapping.json'

# Load the company symbol map
with open(input_file, 'r') as f:
    company_symbols_map = json.loads(f.read())

symbols, names = np.array(list(company_symbols_map.items())).T

# Load the historical stock quotes 
start_date = datetime.datetime(2003, 7, 3)
end_date = datetime.datetime(2007, 5, 4)

#quotes = [quotes_yahoo(symbol, start_date, end_date, asobject=True) 
#                for symbol in symbols]



quotes = pd.read_csv('mpl_finance-master/examples/data/yahoofinance-INTC-19950101-20040412.csv',
                     index_col=0,
                     parse_dates=True,
                     infer_datetime_format=True)

#print(quotes[:10])

# select desired range of dates
df = quotes[(quotes.index >= start_date) & (quotes.index <= end_date)]

print(type(df))

opening = df.loc[:, ['Open', 'Close']]
#print(opening)


# Extract opening and closing quotes
opening_quotes = [quote for quote in quotes]
print(opening_quotes)
quotes.head()


# Extract opening and closing quotes
opening_quotes = np.array(quotes.Open).astype(np.float)
closing_quotes = np.array(quotes.Close).astype(np.float)


# Compute differences between opening and closing quotes 
quotes_diff = closing_quotes - opening_quotes




# Normalize the data 
X = quotes_diff.copy().T
X /= X.std(axis=0)

X = X.reshape(-1, 1)

# Create a graph model 
edge_model = covariance.GraphLassoCV()

# Train the model
with np.errstate(invalid='ignore'):
    edge_model.fit(X)


<class 'pandas.core.frame.DataFrame'>
['Open', 'High', 'Low', 'Close', 'AdjClose', 'Volume']


ValueError: Found array with 1 feature(s) (shape=(2335, 1)) while a minimum of 2 is required by GraphLassoCV.